##### Created on 05/04/2022

##### This notebook is to get the GCM wind vs M relationship for the oceans over 30N to 70N. Will be compared with MERRA2+MAC and PPE outputs. 

##### Updating to retrieve PPE daily data and CESM2 daily data. Used PSL/SST

##### Updating to remove the gcm data at precipitation to check if there is an improvement high M


In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.cm import get_cmap
from readGCMs import read_var_mod
import calendar
from global_land_mask import globe
import glob

In [2]:
#####Constants
Cp = 1004           #J/kg/K
Rd = 287            #J/kg/K
con= Rd/Cp

In [3]:
con

0.2858565737051793

# GCM

In [4]:
modname = ['CESM2','CNRM-CM6-1', 'CanESM5','CESM2-WACCM','E3SM-1-0',
           'HadGEM3-GC31-LL','IPSL-CM6A-LR','NorESM2-LM', 'SAM0-UNICON',
           'ACCESS-CM2','ACCESS-ESM1-5','CNRM-ESM2-1','EC-Earth3',
           'EC-Earth3-Veg','FGOALS-f3-L','GISS-E2-1-G-CC','HadGEM3-GC31-MM',
           'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0','NorCPM1',
           'UKESM1-0-LL']
varname = ['sfcWind', 'tas','psl','pr'] #'sfcWind', 'hfss', 'hfls', 'tas', 'ps', 'psl',
pvarname= ['ta']
conname = ['NCAR', 'CNRM-CERFACS','CCCma', 'NCAR', 'E3SM-Project', 'MOHC', 
           'IPSL', 'NCC', 'SNU','CSIRO-ARCCSS','CSIRO','CNRM-CERFACS',
           'EC-Earth-Consortium','EC-Earth-Consortium','CAS','NASA-GISS','MOHC'
           ,'INM','INM','MPI-M','MRI','NCC','MOHC'] 
ensname = ['r11i1p1f1', 'r1i1p1f2', 'r10i1p1f1', 'r1i1p1f1', 'r1i1p1f1', 
           'r1i1p1f3', 'r10i1p1f1', 'r1i1p1f1', 'r1i1p1f1','r1i1p1f1',
           'r10i1p1f1','r1i1p1f2','r101i1p1f1','r10i1p1f1','r1i1p1f1',
           'r1i1p1f1','r1i1p1f3','r1i1p1f1','r10i1p1f1','r10i1p1f1','r10i1p1f1',
           'r10i1p1f1','r10i1p1f2']
ggname  = ['gn', 'gr','gn', 'gn', 'gr', 'gn', 'gr', 'gn', 'gn','gn', 'gn', 'gr',
           'gr', 'gr', 'gr', 'gn', 'gn', 'gr1', 'gr1', 'gn', 'gr', 'gn', 'gn']

In [5]:
l = 3

In [6]:
try:
    for j in range(l,l+1):
        
        for i in varname: 
            locals()[i+'__'+str(j+1)] = read_var_mod(modn=modname[j], consort=conname[j], varnm=i, cmip='cmip6', exper='historical', ensmem=ensname[j], typevar='day', gg=ggname[j], read_p=False, time1=[2010, 1, 1], time2=[2012, 12, 31])
            print(i)
        for k in pvarname: 
            locals()[k+'__'+str(j+1)] = read_var_mod(modn=modname[j], consort=conname[j], varnm=k, cmip='cmip6', exper='historical', ensmem=ensname[j], typevar='day', gg=ggname[j], read_p=True, time1=[2010, 1, 1], time2=[2012, 12, 31])
            print(k)
        print(modname[l])
        
except IndexError:
    print(str(j+1)+' not available')
    

/glade/work/geethma/research/codes/readGCMs.py:118: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  data = f.variables[varnm][ind]


sfcWind
tas
psl
pr
ta
CESM2-WACCM


In [7]:
i = l

lat  = locals()['sfcWind__'+str(i+1)][2]
lon  = locals()['sfcWind__'+str(i+1)][3]
time = locals()['sfcWind__'+str(i+1)][4]

for j in varname:
    locals()[j+str(i+1)] = locals()[j+'__'+str(i+1)][0]

for k in pvarname:
    locals()[k+str(i+1)] = locals()[k+'__'+str(i+1)][0]
    locals()['lev'+str(i+1)] = locals()['ta__'+str(i+1)][1]

In [8]:
lev4

[masked_array(data=[100000.,  85000.,  70000.,  50000.,  25000.,  10000.,
                      5000.,   1000.],
              mask=False,
        fill_value=1e+20)]

In [8]:
theta_850 = locals()['ta'+str(i+1)][:,1,:,:]*(locals()['psl'+str(i+1)]/85000)**con


In [9]:
M = tas4 - theta_850

In [10]:
M = M[pr4>0]
sfcWind4 = sfcWind4[pr4>0]

In [11]:
tada = sfcWind4
print(np.max(tada), np.min(tada))

27.274075 0.057153072


In [12]:
np.shape(sfcWind4)

(2028503,)

In [13]:
x_lat = np.array(lat)
lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (30)).argmin()])[0]
lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (70)).argmin()])[0]    

x_lon = lon
# lon_ind1 = np.where(x_lon == x_lon.flat[np.abs(x_lon - (-180)).argmin()])[0]
# lon_ind2 = np.where(x_lon == x_lon.flat[np.abs(x_lon - (180)).argmin()])[0]

x_sfcWind = sfcWind4
m_sfcWind = x_sfcWind[:,lat_ind1[0]:lat_ind2[0],:]
lats = lat[lat_ind1[0]:lat_ind2[0]]

x_M = M
m_M = x_M[:,lat_ind1[0]:lat_ind2[0],:]

precip = pr4[:,lat_ind1[0]:lat_ind2[0],:]


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [ ]:
cao = np.array(m_M)
sw  = np.array(m_sfcWind)

In [ ]:
maskm = np.ones((len(time),len(lats),len(lon)))
print(np.shape(maskm))
for a in range(len(lats)):
    for b in range(len(lon)):
        if globe.is_land(lats[a], lon[b])==True:
            maskm[:,a,b] = 0

In [ ]:
plot_CAOI = np.array(np.multiply(maskm,cao))
wind      = np.array(np.multiply(maskm,sw))

In [ ]:
plot_thetai = plot_CAOI.ravel()
plot_windi  = wind.ravel()

In [ ]:
plot_theta = plot_thetai[plot_thetai>-40]
plot_wind  = plot_windi[plot_thetai>-40]

In [ ]:
np.shape(plot_theta)

In [ ]:
np.shape(plot_wind)

In [ ]:
ind = np.argsort(plot_theta)
xx = np.sort(plot_theta)
yy = plot_wind[ind]

In [ ]:
# hb = plt.hexbin(xx, yy, gridsize=40, cmap='Blues')
# plt.ylabel('U10 [m/s]',fontsize='15')
# plt.xlabel('CAOI [K]',fontsize='15')
# plt.title('GCM wind vs M\nfor 30N to 70N ocean only')

In [ ]:
indx = np.isnan(xx*yy)==False

In [ ]:
from scipy import stats
bin_means, bin_edges, binnumber = stats.binned_statistic(xx[indx], yy[indx], 'mean', bins=100)
bin_means_x, bin_edges_x, binnumber_x = stats.binned_statistic(xx[indx], xx[indx], 'mean', bins=100)


In [ ]:
from skmisc.loess import loess
index = np.isnan(bin_means_x*bin_means)==False

plt.plot(bin_means_x[index], bin_means[index])
plt.ylabel('U10 [m/s]',fontsize='15')
plt.xlabel('CAOI [K]',fontsize='15')
# plt.title(modname[l])

In [ ]:
bin_means_x_gcm = bin_means_x[index]
bin_means_gcm   = bin_means[index]

In [ ]:
x_gcm = []
y_gcm = []


In [ ]:
x_gcm.append(bin_means_x_gcm)
y_gcm.append(bin_means_gcm)

# Observations

In [ ]:
import glob
merlist = np.sort(glob.glob('../data_merra/lat_30_70/new/MERRA2_*.nc'))
sfclist = np.sort(glob.glob('../data_merra/lat_30_70/surface/new/MERRA2_*.nc'))
maclist = np.sort(glob.glob('../MACLWP_dailymean/take/wind1deg*.nc4'))

In [ ]:
import netCDF4 as nc
import xarray as xr
p_mer_T   = []
p_mac_w   = []
sfc_mer_T = []
sfc_mer_P = []

for i in range(len(merlist)): #len(merlist)
    d_path = merlist[i]
    data   = nc.Dataset(d_path)
    # print(d_path)
    
    if i==0:
        merlat = data.variables['lat'][:]
        merlon = data.variables['lon'][:]
        merlev = data.variables['lev'][:]
        #shape latitude
        mer_lat = np.flip(merlat)
        mer_lat = np.array(mer_lat)
        #shape longitude
        mer_lon = np.array(merlon)
        
        
    merT   = data.variables['T'][:] #(time, lev, lat, lon)
    mer_T = np.array(merT[:,:,::-1,:])
    p_mer_T.extend(mer_T)
    
temp = np.array(p_mer_T)

for i in range(len(sfclist)): #len(merlist)
    s_path = sfclist[i]
    sdata  = nc.Dataset(s_path)
    # print(d_path)
    
    if i==0:
        sfclat = sdata.variables['lat'][:]
        sfclon = data.variables['lon'][:]
        #shape latitude
        sfc_lat = np.flip(sfclat)
        sfc_lat = np.array(sfc_lat)
        #shape longitude
        sfc_lon = np.array(sfclon)
        
    sfcT   = sdata.variables['TS'][:] 
    sfc_T = np.array(sfcT[:,::-1,:])
    sfc_mer_T.extend(sfc_T)
    
    sfcP   = sdata.variables['SLP'][:] 
    sfc_P = np.array(sfcP[:,::-1,:])
    sfc_mer_P.extend(sfc_P)
    
sfctemp = np.array(sfc_mer_T)
sfcpres = np.array(sfc_mer_P)

for i in range(len(maclist)): #len(maclist)
    ddpath = maclist[i]
    ddata  = nc.Dataset(ddpath)
    macw   = ddata.variables['sfcwind'][:] #(time,lat,lon)
    # print(ddpath)
    
    if i==0:
        maclat = ddata.variables['lat'][:]
        maclon = ddata.variables['lon'][:]
        #shape latitude
        mac_lat = np.array(maclat)
        slat_ind1 = np.where(mac_lat == mac_lat.flat[np.abs(mac_lat - (31)).argmin()])[0]
        slat_ind2 = np.where(mac_lat == mac_lat.flat[np.abs(mac_lat - (71)).argmin()])[0]
        p_mac_lat  = np.array(mac_lat[slat_ind1[0]:slat_ind2[0]])
        #shape longitude
        maclon[maclon > 180] = maclon[maclon > 180]-360
        mac_lon = np.array(maclon)
        p_mac_lon = []
        p_mac_lon.extend(mac_lon[180::])
        p_mac_lon.extend(mac_lon[0:180])
        p_mac_lon = np.array(p_mac_lon)
    n_w = macw[:,slat_ind1[0]:slat_ind2[0],180::]
    new_w = np.append(n_w,macw[:,slat_ind1[0]:slat_ind2[0],0:180],2)
    p_mac_w.extend(new_w)

wind = np.array(p_mac_w)
# wind = np.ma.masked_where(p_mac_w == np.min(p_mac_w), p_mac_w)
# # wind = np.ma.compressed(wind)
# wind = np.ma.masked_where(wind < 5, wind)
# wind = np.ma.compressed(wind)
    

In [ ]:
merlev

In [ ]:
print(sfclat[0],sfclat[-1])
print(sfc_lat[0],sfc_lat[-1])
print(mer_lat[0],mer_lat[-1])
print(p_mac_lat[0],p_mac_lat[-1])


In [ ]:
print(np.shape(temp[:,1,:,:]))
print(np.min(sfcpres))

In [ ]:
len(temp)

In [ ]:
theta_850 = np.array(np.multiply(temp[:,1,:,:], (sfcpres/85000)**(Rd/Cp)))


In [ ]:
p_CAOI = np.array(np.subtract(sfctemp,theta_850))


In [ ]:
maskm = np.ones((len(temp),len(mer_lat),len(mer_lon)))

for a in range(len(mer_lat)):
    for b in range(len(mer_lon)):
        if globe.is_land(mer_lat[a], mer_lon[b])==True:
            maskm[:,a,b] = 0

In [ ]:
plot_CAOI = np.array(np.multiply(maskm,p_CAOI))
plot_wind = np.array(np.multiply(maskm,wind))


In [ ]:
plot_mer_theta = plot_CAOI.ravel()
plot_mac_wind  = plot_wind.ravel()

In [ ]:
print(np.min(plot_mer_theta), np.min(plot_mac_wind))

In [ ]:
ind = np.argsort(plot_mer_theta)
xx = np.sort(plot_mer_theta)
yy = plot_mac_wind[ind]

In [ ]:
x_new = xx[xx<np.max(xx)]
y_new = yy[xx<np.max(xx)]

xx_new = x_new[y_new>0]
yy_new = y_new[y_new>0]

In [ ]:
plt.plot(xx_new,yy_new)

In [ ]:
indx = np.isnan(xx_new*yy_new)==False

In [ ]:
np.where(indx==False)

In [ ]:
bin_means, bin_edges, binnumber = stats.binned_statistic(xx_new, yy_new, 'mean', bins=1000)
bin_means_x, bin_edges_x, binnumber_x = stats.binned_statistic(xx_new, xx_new, 'mean', bins=1000)


In [ ]:
index = np.isnan(bin_means_x*bin_means)==False

In [ ]:

plt.plot(bin_means_x[index], bin_means[index])
plt.ylabel('U10 [m/s]',fontsize='15')
plt.xlabel('CAOI [K]',fontsize='15')
plt.title('observation')

In [ ]:
bin_means_x_obs = bin_means_x[index]
bin_means_obs   = bin_means[index]

In [ ]:
fig= plt.figure(figsize=(5, 6))
plt.plot(bin_means_x_obs, bin_means_obs, label='MACLWP+MERRA2')
plt.plot(bin_means_x_gcm, bin_means_gcm, label='CESM2')
plt.xticks(np.arange(-20,6,5),fontsize='12')
plt.yticks(np.arange(4,13),fontsize='12')
plt.xlim([-20,5])
plt.ylim([3,13])
plt.legend()
plt.ylabel('U10 [m/s]',fontsize='15')
plt.xlabel('CAOI [K]',fontsize='15')
# plt.title('MACLWP wind vs MERRA2 M\nfor 30N to 70N ocean only\n850hPa')

# PPE

In [ ]:
enn = np.arange(201,251)

In [ ]:
ppe_var = ['U10', 'PSL', 'T850','TREFHT']

In [ ]:
for en in enn:
    if en != 175:
        for i in ppe_var: #TREFHT was used since no TS. Should double check this.
            d_path = '/glade/campaign/cgd/projects/ppe/cam_ppe/rerun_PPE_250/PD/PD_timeseries/PPE_250_ensemble_PD.'+f'{en:03d}'+'/atm/hist/cc_PPE_250_ensemble_PD.'+f'{en:03d}'+'.h1.'+str(i)+'.nc'
            data =xr.open_dataset(d_path)

            if en == enn[0]:
                lon  = data.variables['lon'][:]  #(lon: 288) [0.0, 1.25, 2.5, ... 356.25, 357.5, 358.75]
                lat  = data.variables['lat'][:]  #(lat: 192) [-90.0 , -89.057592, -88.115183, ... 88.115183,  89.057592, 90.0]
                time = data.variables['time'][:] #(time: 36)

            locals()[str(en)+'_'+i] = data.variables[i][:]

In [ ]:
x_lat = np.array(lat)
lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (30)).argmin()])[0]
lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (70)).argmin()])[0]
lat_new  = lat[lat_ind1[0]:lat_ind2[0]]

mask = np.ones((1096,len(lat_new),len(lon)))

for a in range(len(lat_new)):
    for b in range(len(lon)):
        if globe.is_land(lat_new[a], lon[b]-180.)==True:
            mask[:,a,b] = 0

for en in enn:
    if en != 175:
        # print(en)
        for i in ppe_var: #
            tmp  = locals()[str(en)+'_'+i]
            tmp2 = tmp[0:1096,lat_ind1[0]:lat_ind2[0],:]
            locals()['MID'+i+'_'+str(en)] = np.multiply(mask,tmp2)

In [ ]:
for en in enn:
    if en != 175:
        locals()['theta_850_'+str(en)] = np.multiply(locals()['MIDT850_'+str(en)],(np.divide(locals()['MIDPSL_'+str(en)],85000))**(Rd/Cp))

In [ ]:
np.shape(theta_850_247)

In [ ]:
for en in enn:
    if en != 175:
        locals()['M_'+str(en)]   = np.array(np.subtract(locals()['MIDTREFHT_'+str(en)],locals()['theta_850_'+str(en)])).reshape(-1)
        locals()['U10_'+str(en)] = np.array(locals()['MIDU10_'+str(en)]).reshape(-1)

In [ ]:
np.max(U10_246)

In [ ]:
from scipy import stats
from skmisc.loess import loess

In [ ]:
fig= plt.figure(figsize=(6, 7))
# plt.plot(bin_means_x_gcm, bin_means_gcm,color='magenta' ,marker='*', linestyle='dashed',
#      markersize=5,linewidth=1, label='CESM2') #marker='*', markersize=12,
# plt.plot(bin_means_x_obs, bin_means_obs,color='blue' ,marker='*', linestyle='dashed',
#      markersize=5,linewidth=1, label='Observations')
for en in enn:
    if en != 175:
        x = locals()['M_'+str(en)]
        y = locals()['U10_'+str(en)]
        ind = np.argsort(x)
        xx = np.sort(x)
        yy = y[ind]

        indx = np.isnan(xx*yy)==False

        bin_means, bin_edges, binnumber = stats.binned_statistic(xx[indx], yy[indx], 'mean', bins=1000)
        bin_means_x, bin_edges_x, binnumber_x = stats.binned_statistic(xx[indx], xx[indx], 'mean', bins=1000)

        index = np.isnan(bin_means_x*bin_means)==False

        # plt.plot(bin_means_x, bin_means)
        l = loess(bin_means_x[index], bin_means[index])
        l.fit()
        pred = l.predict(bin_means_x[index], stderror=True)
        conf = pred.confidence()

        lowess = pred.values


        if en==250:
            plt.plot(bin_means_x[index], lowess, alpha=0.2,label='PPE')

        else:
            plt.plot(bin_means_x[index], lowess, alpha=0.2)
    

# plt.plot(bin_means_x_obs, bin_means_obs, label='MACLWP+MERRA2')
# plt.plot(bin_means_x_gcm, bin_means_gcm, label='CESM2')
# plt.xticks(np.arange(-20,6,5),fontsize='12')
# plt.yticks(np.arange(4,13),fontsize='12')
plt.plot(bin_means_x_gcm, bin_means_gcm,color='magenta' ,marker='*', linestyle='dashed',
     markersize=5,linewidth=1, label='CESM2')
plt.plot(bin_means_x_obs, bin_means_obs,color='blue' ,marker='*', linestyle='dashed',
     markersize=5,linewidth=1, label='Observations')
plt.xlim([-20,5])

plt.legend()
plt.ylabel('U10 [m/s]',fontsize='15')
plt.xlabel('CAOI [K]',fontsize='15')
# plt.title('CAM6 PPE')
# plt.savefig('ppe.png')